In [1]:
import polars as pl

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json

In [2]:
result_df = pl.read_csv('./exp_save/retrain_result.csv')
result_df.head(5), result_df.shape

(shape: (5, 6)
 ┌───────────────┬─────────────────────────────────┬──────────┬──────┬──────────┬──────────┐
 │ dataset       ┆ fea_dim_pair                    ┆ model    ┆ seed ┆ auc      ┆ bce_loss │
 │ ---           ┆ ---                             ┆ ---      ┆ ---  ┆ ---      ┆ ---      │
 │ str           ┆ str                             ┆ str      ┆ i64  ┆ f64      ┆ f64      │
 ╞═══════════════╪═════════════════════════════════╪══════════╪══════╪══════════╪══════════╡
 │ 'placeholder' ┆ 'x                              ┆ ''x'     ┆ 0    ┆ 0.1      ┆ 0.1      │
 │ movielens-1m  ┆ {'genres': 1, 'movie_id': 1, '… ┆ widedeep ┆ 0    ┆ 0.806    ┆ 0.524152 │
 │ movielens-1m  ┆ {'genres': 1, 'movie_id': 1, '… ┆ widedeep ┆ 1    ┆ 0.805873 ┆ 0.52476  │
 │ movielens-1m  ┆ {'genres': 1, 'title': 1, 'use… ┆ widedeep ┆ 0    ┆ 0.796843 ┆ 0.534359 │
 │ movielens-1m  ┆ {'genres': 1, 'title': 1, 'use… ┆ widedeep ┆ 1    ┆ 0.796074 ┆ 0.535249 │
 └───────────────┴─────────────────────────────────┴───

In [16]:
from common.datasets import quick_read_dataset
fs_methods = fs_methods = [
        'no_search',
         'shuffle_dim',
         'shuffle_dim_gradient_no_darts',
         'shuffle_dim_no_darts',
        #  'shuffle_dim_large',
        # 'prune_shuffle_dim',
        # 'autodim',
        'dim_reg',
        # 'sseds',
        'opt_emb',
                  ]

model_list = ['widedeep']

data_list = ['aliccp']
# data_list = ['movielens-1m','aliccp','avazu','criteo']
fs_seed_list = [200,201]
res = {"model":[],"dataset":[],"method":[],'auc':[],'bce_loss':[],'total_params (W)':[],'total_dim':[]}
save_path = './'
for model in model_list:
    for dataset in data_list:
        features, _, unique_values, _ = quick_read_dataset(dataset)
        fea_uniques = {}
        for i,fea in enumerate(features):
            fea_uniques[fea] = unique_values[i]

        
        for fs in fs_methods:
            BCE_loss = []
            AUC = []

            Params = []
            Total_Dim = []
            
            for fs_seed in fs_seed_list:
                
                feature_dim_path = f'./exp_save/fea_dim/{fs}-{dataset}-{model}-{fs_seed}.csv'
                fea_df = pl.read_csv(feature_dim_path).sort(by='fea')
                
                fea_dict = {}
                total_params = 0
                total_dim = 0
                # print(fs)
                # print(fea_df.to_dicts())
                for item in fea_df.to_dicts():
                    if item['dim']>0:
                        fea_dict[str(item['fea'])] = item['dim']
                    total_dim += item['dim']
                    total_params += item['dim'] * fea_uniques[str(item['fea'])]
                
                

                fea_dim_str = str(fea_dict)                    

                auc_loss_np = result_df.filter(
                    pl.col('dataset') == dataset,
                    pl.col('fea_dim_pair') == fea_dim_str,
                    pl.col('model') == model
                )[['auc','bce_loss']].to_numpy()

                AUC.extend(auc_loss_np[:,0].tolist())
                BCE_loss.extend(auc_loss_np[:,1].tolist())

                Params.append(total_params)
                Total_Dim.append(total_dim)

            res['model'].append(model)
            res['dataset'].append(dataset)
            res['method'].append(fs)
            
            res['auc'].append(round(np.mean(AUC),4))
            res['bce_loss'].append(round(np.mean(BCE_loss),4))
            
            res['total_params (W)'].append(round(np.mean(Params)/10000,0))
            res['total_dim'].append(round(np.mean(Total_Dim),4))
            

pl.DataFrame(res).sort('auc',descending=True).filter(
    ~pl.col('auc').is_nan()
).to_pandas()

,model,dataset,method,auc,bce_loss,total_params (W),total_dim
0,widedeep,aliccp,shuffle_dim,0.6590,0.1581,688.0,54.5
1,widedeep,aliccp,shuffle_dim_no_darts,0.6584,0.1582,714.0,56.0
2,widedeep,aliccp,shuffle_dim_gradient_no_darts,0.6583,0.1582,635.0,55.5
3,widedeep,aliccp,no_search,0.6580,0.1582,2497.0,368.0
4,widedeep,aliccp,dim_reg,0.6565,0.1584,681.0,147.5
5,widedeep,aliccp,opt_emb,0.6563,0.1583,1292.0,173.0
